In [ ]:
from FVG import FVGStrategy
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv('/Users/hatim/Desktop/Applied Forecasting/Final Project/Algo Trading/data.csv')

rf = df['Risk Free Rate']
btc = df.drop(columns=['Risk Free Rate'])

strategy = FVGStrategy(lookback_period=10, body_multiplier=1.2, backcandles=50, test_candles=15, upside_scaler=2, downside_scaler=1.2)
signals_df = strategy.generate_signals(btc)

In [ ]:
from evaluate import TraderWithStoploss

evaluator = TraderWithStoploss(initial_cash=100, commission_rate=0.001, risk_free_rate=rf, max_drawdown_threshold=40)
evaluator.backtest_strategy(signals_df)
evaluator.calculate_buy_and_hold(signals_df)

# Print and visualize results
evaluator.print_performance_summary()
evaluator.plot_results()